In [ ]:
from matplotlib.font_manager import win32FontDirectory
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

plt.figure()
fig, ax = plt.subplots(1, 2, figsize = (5, 10))
ax[0].imshow(img1, cmap = 'gray')
ax[0].axis('off')
ax[0].set_title('Oryginalny obraz')

ax[1].imshow(cv2.medianBlur(img1, 5), cmap = 'gray')
ax[1].axis('off')
ax[1].set_title('Po filtrze medianowym')

def adaptive_median(img):

  new_img = np.copy(img)

  s_max = 7
  s_start = 3

  def get_window(pixel_row, pixel_col, size):

    window = np.empty((size, size))
    window[:] = np.nan

    offset = np.floor(size / 2)

    row_min_ind = int(pixel_row - offset)
    row_max_ind = int((pixel_row + offset) + 1)

    col_min_ind = int(pixel_col - offset)
    col_max_ind = int((pixel_col + offset) + 1)

    for img_row_it in range(row_min_ind, row_max_ind):
      for img_col_it in range(col_min_ind, col_max_ind):

        if img_row_it < 0 or img_row_it >= img.shape[0] or img_col_it < 0 or img_col_it >= img.shape[1]:
          continue

        w_row_it = img_row_it - row_min_ind
        w_col_it = img_col_it - col_min_ind

        window[w_row_it, w_col_it] = img[img_row_it, img_col_it]

    return window

  def A(pixel_row, pixel_col, window_size):

    window = get_window(pixel_row, pixel_col, window_size)

    z_min = np.nanmin(window)
    z_max = np.nanmax(window)
    z_med = np.nanmedian(window)

    if z_max > z_med and z_med > z_min:

      #B
      if z_max > img[pixel_row, pixel_col] and img[pixel_row, pixel_col] > z_min:
        return img[pixel_row, pixel_col]
      else:
        return z_med

    else:
      window_size += 2
      if window_size <= s_max:
        return A(pixel_row, pixel_col, window_size)
      else:
        return z_med


  for row_it in range(img.shape[0]):
    for col_it in range(img.shape[1]):

      new_img[row_it, col_it] = A(row_it, col_it, s_start)

  return new_img

plt.figure()
fig, ax = plt.subplots(1, 3, figsize = (10, 20))
ax[0].imshow(img1, cmap = 'gray')
ax[0].axis('off')
ax[0].set_title('Oryginalny obraz')

ax[1].imshow(cv2.medianBlur(img1, 5), cmap = 'gray')
ax[1].axis('off')
ax[1].set_title('Po filtrze medianowym')

ax[2].imshow(adaptive_median(img1), cmap = 'gray')
ax[2].axis('off')
ax[2].set_title('Po filtrze adaptacyjnym')